In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, callbacks
import os

train_dir = '/kaggle/input/kidney-disease-analytica/analytica_PS/train'
test_dir = '/kaggle/input/kidney-disease-analytica/analytica_PS/test'

IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4
MODEL_SAVE_PATH = 'best_kidney_stone_cnn.h5'

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_dataset.class_names
num_classes = len(class_names)
print(f"Found classes: {class_names}")

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
], name='data_augmentation')

model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(32, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes)
])

model.summary()

loss_function = losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

def lr_scheduler(epoch, lr):
    if (epoch + 1) % 7 == 0 and epoch > 0:
        return lr * 0.1
    return lr

lr_callback = callbacks.LearningRateScheduler(lr_scheduler)

checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=MODEL_SAVE_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=['accuracy']
)

print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback, lr_callback]
)
print("Finished Training")

best_val_acc = max(history.history['val_accuracy'])
print(f'Best Validation Accuracy achieved during training: {best_val_acc * 100:.2f}%')

print(f"Best model '{MODEL_SAVE_PATH}' loaded for final evaluation.")
best_model = models.load_model(MODEL_SAVE_PATH)

loss, final_accuracy = best_model.evaluate(test_dataset)

print(f'Final accuracy of the best model on the test images: {final_accuracy * 100:.2f}%')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, callbacks
import os

# --- 1. SETUP AND HYPERPARAMETERS ---


train_dir = '/kaggle/input/kidney-disease-analytica/analytica_PS/train'
test_dir = '/kaggle/input/kidney-disease-analytica/analytica_PS/test'

# Hyperparameter adjustments
IMG_SIZE = 128
BATCH_SIZE = 32
# Increased epochs since we are using Early Stopping
EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4
MODEL_SAVE_PATH = 'best_kidney_stone_cnn_v2.h5'

# --- 2. DATA LOADING ---

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_dataset.class_names
num_classes = len(class_names)
print(f"Found classes: {class_names}")

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


# --- 3. IMPROVED DATA AUGMENTATION AND MODEL ARCHITECTURE ---

# Enhanced data augmentation pipeline
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomContrast(0.1),
], name='data_augmentation')

# Deeper and more robust model architecture
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    layers.Rescaling(1./255),

    # Block 1
    layers.Conv2D(32, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(32, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),

    # Block 2
    layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),

    # Block 3
    layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),
    
    # Block 4
    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2),

    # Classifier Head
    layers.GlobalAveragePooling2D(), # More robust than Flatten for image data
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes)
])

model.summary()

# --- 4. IMPROVED TRAINING SETUP ---

loss_function = losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Define a set of callbacks for smarter training
smart_callbacks = [
    # Stop training if `val_accuracy` doesn't improve for 10 epochs
    callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        verbose=1,
        restore_best_weights=True # Restores model weights from the epoch with the best value
    ),
    # Reduce learning rate if `val_accuracy` plateaus for 3 epochs
    callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.2, # new_lr = lr * factor
        patience=3,
        verbose=1,
        min_lr=1e-6
    ),
    # Save the best model automatically
    callbacks.ModelCheckpoint(
        filepath=MODEL_SAVE_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=['accuracy']
)

# --- 5. TRAINING AND EVALUATION ---

print("Starting training with improved strategy...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    callbacks=smart_callbacks
)
print("Finished Training.")

# The EarlyStopping callback with `restore_best_weights=True` ensures
# the model object already has the best weights, so we can evaluate it directly.
print("\nEvaluating the best model on the test set...")
loss, final_accuracy = model.evaluate(test_dataset)

print(f'Final accuracy of the best model on the test images: {final_accuracy * 100:.2f}%')